1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [ ]:
%pip install -qU pypdf langchain-community

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
%pip install chromadb
%pip install langchain
%pip install sentence-transformers
%pip install docx2txt

In [ ]:
%pip install -qU langchain langchainhub 

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 문서 불러오기
#file_path = "text.pdf"
#loader = PyPDFLoader(file_path)
#pages = loader.load()  # 기본 로드 방식 사용

# 문서 쪼개기
# chunk_size: 문서를 쪼개는 단위
# chunk_overlap: 문서를 쪼개는 단위의 겹치는 부분
# 겹치는 부분을 선언하는 이유는 원하는 문서를 가져올 확률을 높이기 위함
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
#chunks = text_splitter.split_documents(pages)

# load_and_split 사용으로 문서 불러오기와 쪼개기를 한 번에 수행
# 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# 문서 불러오기와 쪼개기를 한 번에 수행
# XXX 현재 pdf에서는 원하는 데이터를 가져오지 못함
# pdf에서의 원하는 데이터는 사진 형식의 표로 들어가 있기 때문.
# gpt에서 표를 이해하는 방식은 모델에 따라 다르지만 openAI는 markdown으로 이해하기에 markdown으로 변환이 필요하다.
file_path = "text.docx"
loader = Docx2txtLoader(file_path)
document_list = loader.load_and_split(text_splitter)


# 임베딩
load_dotenv()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
%pip install -qU langchain-pinecone 

In [ ]:
import os
from dotenv import load_dotenv

from langchain_pinecone import PineconeVectorStore

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")

database = PineconeVectorStore(
    index_name="tax-collection",
    embedding=embeddings,
    pinecone_api_key=pinecone_api_key
)
database.from_documents(document_list)



In [14]:
# 유사도 검색
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
retrieved_docs

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

# 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달
# 페르소나를 전달하면 답변이 더 잘나온다고 한다.
prompt = f"""[Identity]
- 당신은 한국의 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 대답해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""
response = llm.invoke(prompt)

In [ ]:
response.content

In [ ]:
from langchain import hub

# question-answering prompt
# 기존에 이미 있는 프롬프트 활용
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


In [17]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message